In [37]:
from bs4 import BeautifulSoup
import requests
from itertools import chain
import pandas as pd
import re
import numpy as np

In [38]:
dfz = pd.read_excel('cities.xlsx')

In [39]:
cli = dfz['city'].tolist()

In [40]:
cli

['Tirana',
 'Salzburg',
 'Graz',
 'Linz',
 'Vienna',
 'Brussels',
 'Gent',
 'Antwerp',
 'Leuven',
 'Liège',
 'Sofia',
 'Zagreb',
 'Dubrovnik',
 'Prague',
 'Brno',
 'Copenhagen',
 'Odense',
 'Tallinn',
 'Helsinki',
 'Strasbourg',
 'Bordeaux',
 'Dijon',
 'Rennes',
 'Reims',
 'Rouen',
 'Paris',
 'Montpellier',
 'Limoges',
 'Metz',
 'Toulouse',
 'Lille',
 'Nantes',
 'Cannes',
 'Marseille',
 'Nice',
 'Grenoble',
 'Lyon',
 'Saint-Étienne',
 'Le Havre',
 'Karlsruhe',
 'Mannheim-Heidelberg',
 'Stuttgart',
 'Munich',
 'Nuremberg',
 'Berlin',
 'Bremen',
 'Hamburg',
 'Frankfurt',
 'Hannover',
 'Aachen',
 'Bochum',
 'Bonn',
 'Cologne',
 'Dortmund',
 'Duisburg',
 'Düsseldorf',
 'Essen',
 'Wuppertal',
 'Dresden',
 'Leipzig',
 'Kiel',
 'Athens',
 'Thessaloniki',
 'Budapest',
 'Reykjavik',
 'Dublin',
 'Bologna',
 'Modena',
 'Parma',
 'Trieste',
 'Rome',
 'Milan',
 'Torino',
 'Florence',
 'Padova',
 'Venice',
 'Verona',
 'Riga',
 'Vilnius',
 'Luxembourg',
 'Skopje',
 'Chrisinau',
 'Amsterdam',
 'Eindho

In [146]:
def create_frame(ls):
    
    total = pd.DataFrame()
    
    for city in ls:
        new =  []

        page = requests.get("https://www.numbeo.com/cost-of-living/in/" + city)
        soup = BeautifulSoup(page.content)
        data = soup.find_all("table",{"class":"data_wide_table new_bar_table"})
        data_cost = soup.find_all("span",{"class":"emp_number"})
        
        try:
            cost_index = re.search('>(.*)\<', str(data_cost[-1])).group(1)
        except IndexError:
            pass
        
        l = [item.text.split('€') for item in data]
        newlist = list(chain(*l))

        new.append('City' + '  '  + city)
        new.append('Cost_index' + '  ' + cost_index)

        for i in range(len(newlist)):
            try:
                new.append(re.search('\n(.*)\xa0', newlist[i]).group(1))
            except AttributeError as e:
                pass
            
        if len(new) != 56:
            pass
        
        else:
            for i in range(len(new)):
                total = total.append(pd.DataFrame({'':new[i].split('  ')[0], 'Cost':new[i].split('  ')[1]},index=[0]), ignore_index=True)
            
            headers = total.T.iloc[0]
            headers = pd.DataFrame(headers[:56])
            
    values = []
    n = len(total)/56
    df_list = np.vsplit(total, n)

    for i in df_list:
        values.append(i['Cost'].tolist())
    
    v = pd.DataFrame(values)
    zz = pd.DataFrame(headers).T
    xx = pd.concat([zz,v])
    new_header = xx.iloc[0]
    xx = xx[1:]
    xx.columns = new_header
    
    final = xx.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    final = final.drop(['Water (0.33 liter bottle)'], axis = 1)
    
    zz = final['City']
    z5 = final.loc[:, final.columns != 'City']
    z5 = z5.replace(',', '', regex=True)
    z39 = z5.apply(pd.to_numeric, errors='coerce')
    df = pd.concat([zz, z39],axis=1)
    
            
    return df

In [147]:
df = create_frame(cli)

In [148]:
df

,City,Cost_index,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Cappuccino (regular),Coke/Pepsi (0.33 liter bottle),"Milk (regular), (1 liter)",...,"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax)
0,Salzburg,72.56,11.5,50.0,7.60,4.00,3.95,3.11,2.80,1.08,...,32.67,74.17,112.80,754.44,648.75,1508.33,1191.67,6625.00,4560.00,1912.50
1,Graz,72.51,10.0,45.0,7.90,3.80,3.50,3.05,2.43,1.15,...,28.97,83.62,119.69,589.26,440.75,1024.12,803.75,3691.79,2907.65,1911.25
2,Linz,72.66,14.0,60.0,8.00,4.20,4.20,3.25,2.92,1.12,...,35.29,91.82,115.25,666.52,545.91,1367.62,997.14,4070.59,2994.12,1977.73
3,Vienna,65.07,10.0,50.0,7.50,4.00,4.00,3.32,2.83,1.09,...,33.12,83.68,116.15,876.59,640.75,1615.67,1168.67,7138.82,4251.28,2089.24
4,Brussels,69.73,15.0,60.0,8.68,4.00,4.00,3.12,2.57,0.98,...,35.46,78.33,116.11,845.87,698.97,1422.00,1190.00,3322.62,2576.19,2312.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Bilbao,63.57,12.0,50.0,8.00,2.75,3.00,1.52,2.09,1.03,...,39.00,99.29,126.25,812.50,654.67,1246.43,928.54,4436.65,3100.00,1553.43
61,Madrid,58.93,12.0,50.0,8.00,3.00,3.50,1.86,1.93,0.83,...,28.98,74.49,91.80,992.79,732.95,1734.91,1196.46,5170.73,3140.47,1639.67
62,Granada,55.56,10.0,31.0,8.00,2.50,3.00,1.64,2.04,0.85,...,31.00,73.15,93.33,511.19,405.83,835.00,603.61,2291.67,1783.33,1287.50
63,Pamplona,55.56,10.5,37.5,7.00,3.00,3.50,1.30,1.46,0.70,...,31.25,54.00,100.00,664.29,525.00,1000.00,790.00,3848.33,1800.00,1325.00
